In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important }</style>"))

In [2]:
import joblib
import pandas as pd
import numpy as np
from influxdb import DataFrameClient
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [3]:
scaler = joblib.load("../Models/rolling_statistics_nn/scaler")

In [4]:
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[68]),
    layers.Dense(2)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

In [5]:
model.load_weights("../Models/rolling_statistics_nn/cp.ckpt")

In [6]:
client = DataFrameClient(host='localhost', port=8086)
client.get_list_database()

[{'name': '_internal'}, {'name': 'Megallen2'}]

In [7]:
client.switch_database('Megallen2')

In [8]:
res = client.query("SELECT * FROM OnlineRSSI WHERE time > now() - 5s")
if res == {}:
    raise Exception("No data incoming")
res = res["OnlineRSSI"]

In [9]:
B1 = res[res["beacon"] == "0117C55D14E4"]

In [10]:
B1

,beacon,rssi,scanner
2019-12-06 09:32:54.651000+00:00,0117C55D14E4,-73,CD4533FFC0E1
2019-12-06 09:32:54.659000+00:00,0117C55D14E4,-62,DF231643E227
2019-12-06 09:32:54.674000+00:00,0117C55D14E4,-75,E96AF2C858BA
2019-12-06 09:32:54.689000+00:00,0117C55D14E4,-76,F1307ECB3B90
2019-12-06 09:32:54.693000+00:00,0117C55D14E4,-76,D2B6503554D7
...,...,...,...
2019-12-06 09:32:59.479000+00:00,0117C55D14E4,-67,CD4533FFC0E1
2019-12-06 09:32:59.493000+00:00,0117C55D14E4,-72,F1307ECB3B90
2019-12-06 09:32:59.554000+00:00,0117C55D14E4,-69,E13B805C6CB0
2019-12-06 09:32:59.554000+00:00,0117C55D14E4,-85,E43355CA8B96


In [11]:
B1 = B1.pivot(columns="scanner", values="rssi").rename_axis(None, axis=1)
B1

,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
2019-12-06 09:32:54.651000+00:00,NaN,-73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 09:32:54.659000+00:00,NaN,NaN,NaN,NaN,NaN,-62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 09:32:54.674000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 09:32:54.689000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.0,NaN,NaN,NaN,NaN
2019-12-06 09:32:54.693000+00:00,NaN,NaN,-76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-06 09:32:59.473000+00:00,NaN,-70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 09:32:59.479000+00:00,NaN,-67.0,NaN,NaN,NaN,-62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 09:32:59.493000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-72.0,NaN,NaN,NaN,NaN
2019-12-06 09:32:59.554000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,-69.0,-85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
scanners = [
    'C400A2E19293', 'CD4533FFC0E1', 'D2B6503554D7', 'DB8B36A69C56', 'DD697EA75B68', 'DF231643E227', 
    'E13B805C6CB0', 'E43355CA8B96', 'E6D9D20DD197', 'E8FD0B453DC4', 'E96AF2C858BA', 'EC72840D9AD3', 
    'F1307ECB3B90', 'F1EDAF28E08A', 'F69A86823B96', 'FB2EE01C18CE', 'FDAE5980F28C'
]
B1 = B1.reindex(columns=scanners)
B1

,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
2019-12-06 09:32:54.651000+00:00,NaN,-73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 09:32:54.659000+00:00,NaN,NaN,NaN,NaN,NaN,-62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 09:32:54.674000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 09:32:54.689000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-76.0,NaN,NaN,NaN,NaN
2019-12-06 09:32:54.693000+00:00,NaN,NaN,-76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-06 09:32:59.473000+00:00,NaN,-70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 09:32:59.479000+00:00,NaN,-67.0,NaN,NaN,NaN,-62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-06 09:32:59.493000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-72.0,NaN,NaN,NaN,NaN
2019-12-06 09:32:59.554000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,-69.0,-85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
data = B1.rolling(50, min_periods=1).agg(["mean", "std", "median", "count"]).ffill().fillna(-100)

In [14]:
data = scaler.transform(data)

In [15]:
pred = model.predict(data)

In [16]:
np.expand_dims(pred.mean(axis=0), axis=0)

array([[24.798536, 11.101005]], dtype=float32)